In [1]:
import tensorflow as tf
# from tensorflow.examples.tutorials.mnist import input_data
# mnist = input_data.read_data_sets("MNIST_data/", one_hot=True)

In [2]:
import numpy as np
import pickle

In [3]:
data_files = ['dev', 'test', 'train']

data_path = "./data/%s_dataset.json"
x_file = './data/op_data/%s_op_X.pkl'
y_file = './data/op_data/%s_op_Y.pkl'
idx_file = './data/op_data/%s_op_idx.pkl'

In [4]:
op_n_input = 1200
op_n_hidden1 = 512
op_n_hidden2 = 256
op_n_hidden3 = 64
op_n_output = 4

op_learning_rate = 1e-4
op_n_iteration = 31
op_batch_size = 128
op_dropout = 0.1

op_X = tf.placeholder("float", [None, op_n_input])
op_Y = tf.placeholder("float", [None, op_n_output])
# keep_prob = tf.placeholder(tf.float32) # used to control the dropout rate

op_weights = {
    'w1': tf.Variable(tf.truncated_normal([op_n_input, op_n_hidden1], stddev=0.1)),
    'w2': tf.Variable(tf.truncated_normal([op_n_hidden1, op_n_hidden2], stddev=0.1)),
    'w3': tf.Variable(tf.truncated_normal([op_n_hidden2, op_n_hidden3], stddev=0.1)),
    'out': tf.Variable(tf.truncated_normal([op_n_hidden3, op_n_output], stddev=0.1))
}

op_biases = {
    'b1': tf.Variable(tf.constant(0.1, shape=[op_n_hidden1])),
    'b2': tf.Variable(tf.constant(0.1, shape=[op_n_hidden2])),
    'b3': tf.Variable(tf.constant(0.1, shape=[op_n_hidden3])),
    'out': tf.Variable(tf.constant(0.1, shape=[op_n_output]))
}

op_layer_1 = tf.nn.tanh(tf.add(tf.matmul(op_X, op_weights['w1']), op_biases['b1']))
op_layer_2 = tf.nn.tanh(tf.add(tf.matmul(op_layer_1, op_weights['w2']), op_biases['b2']))
op_layer_3 = tf.nn.relu(tf.add(tf.matmul(op_layer_2, op_weights['w3']), op_biases['b3']))
# layer_drop = tf.nn.dropout(layer_3, keep_prob)
op_output_layer = tf.nn.leaky_relu(tf.add(tf.matmul(op_layer_3, op_weights['out']), op_biases['out']))

op_cross_entropy = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(labels=op_Y, logits=op_output_layer))
op_train_step = tf.train.AdamOptimizer(op_learning_rate).minimize(op_cross_entropy)

op_correct_pred = tf.equal(tf.argmax(op_output_layer, 1), tf.argmax(op_Y, 1))
op_accuracy = tf.reduce_mean(tf.cast(op_correct_pred, tf.float32))

op_init = tf.global_variables_initializer()
op_sess = tf.Session()
op_sess.run(op_init)

In [5]:
def get_data(dt):
    with open(y_file%dt, 'rb') as fy:
        _Y = pickle.load(fy)

    with open(x_file%dt, 'rb') as fx:
        _X = pickle.load(fx)
    return (_X, _Y)

In [6]:
op_dev_X, op_dev_Y = get_data('train')

In [7]:
def get_batch(idx, batch_size):
    lw = idx * batch_size
    hs = lw + batch_size
    if hs < len(op_dev_X):
        return (op_dev_X[lw:hs][:], op_dev_Y[lw:hs][:])
    else:
        return (op_dev_X[lw:][:], op_dev_Y[lw:][:])

In [8]:
op_n_batches = len(op_dev_X)//op_batch_size

In [9]:
len(op_dev_X)

55499

In [10]:
# train on mini batches
for i in range(op_n_iteration):
    for j in range(op_n_batches+1):
        (op_batch_x, op_batch_y) = get_batch(j, op_batch_size)
        op_sess.run(op_train_step, feed_dict={op_X: op_batch_x, op_Y: op_batch_y})
        
        # print loss and accuracy (per minibatch)
    if i % 5 == 0:
        op_minibatch_loss, op_minibatch_accuracy = op_sess.run([op_cross_entropy, op_accuracy], feed_dict={op_X: op_batch_x, op_Y: op_batch_y})
        print("Iteration", str(i), "\t| Batch", str(j), "\t| Loss =", str(op_minibatch_loss), "\t| Accuracy =", str(op_minibatch_accuracy))

Iteration 0 	| Batch 433 	| Loss = 0.35148177 	| Accuracy = 0.8933333
Iteration 5 	| Batch 433 	| Loss = 0.24514925 	| Accuracy = 0.93333334
Iteration 10 	| Batch 433 	| Loss = 0.17745362 	| Accuracy = 0.96
Iteration 15 	| Batch 433 	| Loss = 0.14896502 	| Accuracy = 0.96
Iteration 20 	| Batch 433 	| Loss = 0.1355629 	| Accuracy = 0.96
Iteration 25 	| Batch 433 	| Loss = 0.12456276 	| Accuracy = 0.96
Iteration 30 	| Batch 433 	| Loss = 0.118668914 	| Accuracy = 0.96


In [12]:
tf.train.Saver().save(op_sess, './checkpoints/op/training.ckpt')

'./checkpoints/op/training.ckpt'

In [ ]:
test_X, test_Y = get_data('test')

test_accuracy = sess.run(accuracy, feed_dict={X: test_X, Y: test_Y})
print("\nAccuracy on test set : ", test_accuracy)